In [8]:
import cv2
import glob
import numpy as np
import math

In [ ]:
def find_circles(thresh_frame):
    contours, hierarchy = cv2.findContours(thresh_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#
    contours_cirles = []
    for con in contours:
        perimeter = cv2.arcLength(con, True)
        area = cv2.contourArea(con)
        if perimeter == 0 or area>500:
            return []
        circularity = 4*math.pi*(area/(perimeter*perimeter))
        
        if 0.8 < circularity < 1.2:
            contours_cirles.append(con)
    return contours_cirles

# Using Red color channels from RGB (issue when there is lots of white spots where intensity for all RGB values are equally high)

In [196]:
cap = cv2.VideoCapture(r"../data/stable_background.mp4")
while True:
    ret, stable_frame = cap.read()           
    if not ret:
        break
    stable_frame_blurred = cv2.GaussianBlur(stable_frame,(11,11),0)
    img_r = stable_frame_blurred[:,:,0]
    threshold = int(img_r.max()*0.95)
    ret,thresh_frame = cv2.threshold(img_r,threshold,255,cv2.THRESH_BINARY)
    circles = find_circles(thresh_frame)
    for c in circles:
        M = cv2.moments(c)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            cv2.drawMarker(stable_frame, (cx, cy), color=[0, 255, 0], thickness=2, markerType= cv2.MARKER_TILTED_CROSS, line_type=cv2.LINE_AA,markerSize=7)



    #cv2.drawContours(stable_frame, circles, 0, (0,255,0), 3)
    cv2.imshow("frame", stable_frame)
    cv2.imshow("filtered", thresh_frame)
    if cv2.waitKey(10) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        print ("Exiting program")
cv2.destroyAllWindows()

Exiting program


# Using LAB color spaces (A channel)

In [219]:
cap = cv2.VideoCapture(r"../data/laser-treatment.mp4")
while True:
    ret, stable_frame = cap.read()           
    if not ret:
        break
    lab = cv2.cvtColor(stable_frame, cv2.COLOR_RGB2LAB)
    L,A,B=cv2.split(lab)
    stable_frame_blurred = cv2.GaussianBlur(A,(15,15),0)
    threshold = int(stable_frame_blurred.max()*0.95)
    #print(f"thresh: {threshold}")
    ret,thresh_frame = cv2.threshold(stable_frame_blurred,threshold,255,cv2.THRESH_BINARY)
    circles = find_circles(thresh_frame)
    for c in circles:
        M = cv2.moments(c)
        if M['m00'] != 0:
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            cv2.drawMarker(stable_frame, (cx, cy), color=[0, 255, 0], thickness=2, markerType= cv2.MARKER_TILTED_CROSS, line_type=cv2.LINE_AA,markerSize=7)

    #cv2.drawContours(stable_frame, circles, 0, (0,255,0), 3)
    cv2.imshow("stable_frame", stable_frame)
    cv2.imshow("thresh_frame", thresh_frame)
    cv2.imshow("A", A)
    if cv2.waitKey(10) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        print ("Exiting program")
cv2.destroyAllWindows()

2831.5
3031.5
2920.0
2988.5
3059.5
3127.5
3249.5
3283.0
1.0
3486.0
3425.5
3492.0
3165.0
3178.0
3289.5
3335.0
2783.5
2778.0
3464.0
4266.0
52.0
4830.5
3413.5
3370.0
3573.0
3416.0
3552.5
3636.5
3657.0
3192.0
3020.5
3119.5
3754.5
3807.5
3247.5
2466.5
2574.0
2520.5
2544.0
2481.0
2514.5
2522.5
2644.5
2446.5
2470.5
2409.5
2413.5
44.5
29.0
45.0
33.0
29.0
33.0
31.0
80.0
22.0
26.5
29.0
25.5
22.0
26.0
62.0
24.0
26.0
21.0
20.5
23.5
23.5
29.5
19.0
24.5
25.0
29.0
25.5
26.5
30.5
25.5
26.0
11.5
33.0
77.5
65.0
20.0
20.5
105.0
18.5
170.0
7.5
168.0
7.5
165.5
171.0
4.5
159.5
5.5
163.0
98.5
20.5
101.0
28.5
30.5
29.5
34.0
24.5
26.5
12.0
21.5
28.5
23.5
18.5
1.5
23.5
24.0
25.0
23.0
24.5
27.0
30.0
42.0
84.5
27.5
26.5
21.5
24.0
24.5
25.0
27.5
30.0
40.5
29.5
37.0
49.0
35.5
37.5
37.0
36.0
40.5
33.0
39.0
38.0
33.5
31.5
43.5
43.0
52.0
46.5
42.0
41.5
50.5
43.0
43.0
56.0
83.0
65.0
44.0
48.5
42.5
43.5
38.5
39.0
48.0
66.0
37.0
37.0
38.5
52.0
69.5
34.0
27.0
30.0
24.5
28.0
32.0
30.0
30.0
30.5
31.0
39.5
32.0
35.0
30.0
31.

# Using HSV to filter red

In [202]:
cap = cv2.VideoCapture(r"../data/unstable_background.mp4")
while True:
    ret, frame = cap.read()           
    if not ret:
        break
    
    
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    frame_blurred = cv2.GaussianBlur(frame,(5,5),0)

    # Define lower and upper HSV boundaries for the color blue
    lower_red_1 = np.array([0, 50, 150])
    upper_red_1 = np.array([10, 255, 255])


    # Create a mask with cv2.inRange to detect red colors
    mask = cv2.inRange(frame_blurred, lower_red_1, upper_red_1)

    
    # Use bitwise AND to extract the red color from the original image

    circles = find_circles(mask)
    cv2.drawContours(frame, circles, 0, (0,255,0), 3)
    
    cv2.imshow("mask", mask)
    cv2.imshow("original", frame)
    if cv2.waitKey(10) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        print ("Exiting program")
cv2.destroyAllWindows()

Exiting program


In [200]:
cv2.destroyAllWindows()

# LAB color space seems to have worked best for red colored lasers